# Data used in Convotate
This notebook outlines steps in data cleaning in the development of Convotate. Our initial sequence dump was taken from 23k taxonomically dereplicated/diverse genomes based on the taxIDs given by GTDB. Basic initial filtering, which consisted of removal of duplicate sequences and removal of sequences <30AA long, was done in bash. This provides example code/data for how we dealt with very rare subsystems, and the files/code used to merge the ontology. We have not provided the full sequence dumps as they are over 30GB in compressed format, however, you may access/see:
1. genome ID dump from GTDB release 83 (.tree), which lists NCBI tax IDs: https://data.ace.uq.edu.au/public/gtdb/data/releases/release83/83.0/
2. all raw data from PATRIC (www.patricbrc.org), ftp: ftp://ftp.patricbrc.org
3. our final cleaned data (~3.34GB compressed tar.bz2 file), consisting of ~14 million training, ~4 million test sequences, and ontology/label files, here: https://www.dropbox.com/s/baz9dq3obhcz5lq/FinalDataFiles.tar.bz2?dl=0

Required packages:
pandas, numpy, itertools, python-levenshtein


In [2]:
import pandas as pd
import numpy as np
from itertools import combinations

import Levenshtein
import csv
import glob

## Fuzzy Match for diverse random sequence selection
About 200 subsystems did not have enough data (1000 sequences or more). We searched for sequences assigned to those subsystems in the full PATRIC sequence dump, removed duplicates, and then used string fuzzy match to get a 'diverse' random selection of these as demonstrated below. If there were fewer than 1000 sequences in a subsystem for the complete PATRIC back end, we excluded it from the study. This is one (very small) example with the subsystem 'Cytolethal Distending Toxins' - only 6000 sequences in the dump. The data file contains columns of: PATRIC tax id (maps to NCBI id), protein ID, ontology columns, AA sequences.  

In [12]:
for filepath in glob.iglob('./data/fuzzy_match/*'):
    print(filepath,'is computing')
    
    # load sequences
    sequences = pd.read_csv(filepath, sep='\t',header=None,names=['feature.patric_id', 'feature.aa_sequence'],usecols=[1,6])
    aa_string1=sequences.iloc[0][1]
    figID1=sequences.iloc[0][0]
    
    ## calc Levenshtein distance and append to the dataframe
    print('calculating Levenshtein distances')
    distance = []
    for index, row in sequences.iterrows():
        distance.append(Levenshtein.ratio(aa_string1, row["feature.aa_sequence"]))
    
    sequences['ldistance']=distance
    
    ## set up sampling - levenshtein bins 
    # create uniform bin size (20 bins in total)
    bins=np.linspace(min(sequences['ldistance']),max(sequences['ldistance']),21).tolist()
    labels = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    sequences['binned'] = pd.cut(sequences['ldistance'], bins=bins, labels=labels)
    print('the bins are',bins)
    
    ## figure out how many samples to take per bin, create sampling function

    nbin_counts=sequences['binned'].value_counts().to_frame()
    nbins_large=nbin_counts[nbin_counts> 500].count()
    nsamp=int(5000/(nbins_large))
    print('The bin counts are','\n',nbin_counts)
    
    def balanced_randsample(s):
        try:
            return s.sample(min(len(s), nsamp))
        except: pass
    
    ## take sample
    balanced_sequence_set=sequences.groupby('binned')['feature.patric_id'].apply(lambda s: balanced_randsample(s))
    balanced_sequence_set=balanced_sequence_set.to_frame().reset_index(level=0, drop=True).reset_index()
    
    ## look up patric ID indices in aa sequence frame, return list to write to file
    selected_sequences=[]
    for row in balanced_sequence_set['index']:
        get_elements=sequences.iloc[row][0:2]
        elements=get_elements['feature.patric_id'],get_elements['feature.aa_sequence']
        selected_sequences.append(elements)
    
    ## write sample to file
    print('writing',len(selected_sequences),'selected sequences to file for',filepath)

    # with open('./data/fuzzy_match/new_sequences.csv','w') as f:
    #     writer = csv.writer(f)
    #     writer.writerow(['feature.patric_id', 'feature.aa_sequence'])
    #     writer.writerows(selected_sequences)

./data/fuzzy_match/Cytolethal distending toxins is computing
calculating Levenshtein distances
the bins are [0.15181518151815182, 0.19422442244224422, 0.23663366336633662, 0.27904290429042905, 0.3214521452145215, 0.36386138613861385, 0.4062706270627063, 0.44867986798679865, 0.4910891089108911, 0.5334983498349835, 0.5759075907590758, 0.6183168316831683, 0.6607260726072608, 0.7031353135313532, 0.7455445544554455, 0.7879537953795379, 0.8303630363036303, 0.8727722772277227, 0.9151815181518153, 0.9575907590759076, 1.0]
The bin counts are 
     binned
10    2558
6     1822
11     638
5      301
3      203
8      126
4      119
20      95
9       80
12      71
2       46
7       37
19      13
1        3
14       2
13       1
15       1
16       1
17       0
18       0
writing 5069 selected sequences to file for ./data/fuzzy_match/Cytolethal distending toxins


## Merging Subsystems and Ontology
As the SEED is a manually curated system, some Subsystems have significant overlap - or are even subsets of another. Subsystems are created with groups of 'roles' (EC assignments or the like). We assessed roles per subystem and merged if there was >=80% overlap.

#### Import the files and do some preprocessing
Import the information about which roles are in which subsystems (manually taken from the PATRIC website due to API issues). Convert roles IDs to integers.

In [36]:
# Import subsystems and complete list of roles
ssRoleNameDf=pd.read_csv('./data/ontology_merging/ss_rolenames_import', sep='\t', 
                         header=None).rename(columns={0:'Subsystem Name',1:'Roles'})
roleUnique=pd.read_csv('./data/ontology_merging/all_roles', sep='\t', 
                       header=None).rename(columns={0:'roleName'})

# Create dictionary linking all roles to an integer uid
RoleDictNames = {}
RoleDictNames=dict(sorted(ssRoleNameDf.values.tolist()))

RoleDict = {}
for i in range(len(roleUnique)):
    RoleDict[roleUnique.loc[i,:].values[0]] = i

    
# Print the dataframe and the first role
display(ssRoleNameDf.head())     

print('The first role uid is:',RoleDict.get('(2E,6E)-farnesyl diphosphate synthase (EC 2.5.1.10)'))

,Subsystem Name,Roles
0,D-Galacturonate and D-Glucuronate Utilization,• Mannonate dehydratase (EC 4.2.1.8) • D-manno...
1,Flagellum,• Flagellar motor rotation protein MotB • Flag...
2,Type 4 secretion and conjugative transfer,• Inner membrane protein of type IV secretion ...
3,"Ribosomal proteins, single-copy",• LSU ribosomal protein L11p (L12e) • SSU ribo...
4,Folate Biosynthesis,• Serine hydroxymethyltransferase (EC 2.1.2.1)...


The first role uid is: 0


In [33]:
## Convert the subsystem-role dataframe to a dictionary with subsystem names as a key
## and roles as values (in a list). Clean up the bullets, whitespaces etc and then convert to uids.

# Create the dictionary
subsystemRoleName=dict(sorted(ssRoleNameDf.values.tolist()))

# Clean up the formatting and then convert to uids
for key in subsystemRoleName:
    s1 = subsystemRoleName.get(key)
    units = s1.split('•')

    for i in range(len(units)):
        units[i] = units[i].strip()

    while('' in units) : 
        units.remove('')

    for i in range(len(units)):
        units[i]=RoleDict.get(units[i])    
    units=sorted(units)
    subsystemRoleName.update({key: units})

### Merge the similar subsystems
Merge subsystems if there is >=80% similarity between their roles 

In [56]:
## Create a list of classes which need to be merged together, based on their role overlaps

mergeClasses=[]

for key1, key2 in combinations(subsystemRoleName.keys(), r = 2):
    subsystem1_rolelist = subsystemRoleName.get(key1)
    subsystem2_rolelist = subsystemRoleName.get(key2)
    is_subset = []
   
    if len(subsystem1_rolelist) <= len(subsystem2_rolelist):
        if len(subsystem1_rolelist) > 1:
            is_subset=list(map(lambda each: each in subsystem2_rolelist, subsystem1_rolelist))
        else:
            continue
    else:
        if len(subsystem2_rolelist) > 1:
            is_subset=list(map(lambda each: each in subsystem1_rolelist, subsystem2_rolelist))
        else:
            continue 
    intersec = np.int0(is_subset)
    
    if sum(intersec)/len(intersec) > 0.79:
        mergeClasses.append([key1,key2])
        
print('The first 3 subsystems which need to be merged are: \n',mergeClasses[0:3])

The first 3 subsystems which need to be merged are: 
 [['2-oxoglutarate dehydrogenase', 'Lipoylated proteins'], ['2-oxoglutarate dehydrogenase', 'TCA Cycle'], ['ATP-dependent Nuclease', 'DNA repair, bacterial RecBCD pathway']]


In [44]:
## list merge function from stackoverflow
def merge(lists, results=None):
    if results is None:
        results = []

    if not lists:
        return results

    first = lists[0]
    merged = []
    output = []

    for li in lists[1:]:
        for i in first:
            if i in li:
                merged = merged + li
                break
        else:
            output.append(li)

    merged = merged + first
    results.append(list(set(merged)))

    return merge(output, results)

Because some cases required >2 subsystems to be merged together (i.e. there are three or more  overlaps). we then recursively merge the pairs listed in the output (any common elements mean they end up in the same bin)

In [54]:
fullyMergedClasses=merge(mergeClasses)
fullyMergedClasses=merge(fullyMergedClasses)

These were then appended to the appropriate row as a new column 'Subsystem Merged' in the ontology. We used a delimiter of '!' between concatenated subsystems to prevent confusion arising from the fact commas were in some labels, and keeping substring quotes was causing issues with automatic text parsing. 